In [ ]:
import cv2
from google.colab.patches import cv2_imshow

In [ ]:
img = cv2.imread('/content/car.jpeg')

In [ ]:
cv2_imshow(img)
cv2.waitKey(0)

In [ ]:
img1 = cv2.imread('/content/cat.jpg')


In [ ]:
cv2_imshow(img1)
cv2.waitKey(0)

In [ ]:
classNames = []
classFile = '/content/coco.names'
with open(classFile, 'rt') as f:
  classNames = f.read().rstrip('\n').split('\n')

In [ ]:
print(classNames)

In [ ]:
configPath = '/content/ssd_mobilenet_v3_large_coco_2020_01_14.pbtxt'
weightsPath = '/content/frozen_inference_graph.pb'

In [ ]:
net = cv2.dnn_DetectionModel(weightsPath, configPath)
net.setInputSize(320, 320)
net.setInputScale(1.0/127.5)
net.setInputMean((127.5, 127.5, 127.5))
net.setInputSwapRB(True)

In [ ]:
classIds, confs, bbox = net.detect(img, confThreshold=0.7)

In [ ]:
print(classIds, bbox)

In [ ]:
for classId, confidence, box in zip(classIds.flatten(), confs.flatten(), bbox):
    x, y, w, h = box
    cv2.rectangle(img, (x, y), (x+w, y+h), color=(0,255,0), thickness=3)
    cv2.putText(img, classNames[classId-1], (x+10, y+30), cv2.FONT_HERSHEY_COMPLEX, 2, (0, 255, 0), 2)

In [ ]:
cv2_imshow(img)

In [ ]:

def detect_objects_in_images(image_paths, model_path, config_path):
    """Detects objects in multiple images using a pre-trained model.

    Args:
        image_paths: A list of paths to the input images.
        model_path: Path to the pre-trained model weights.
        config_path: Path to the model configuration file.

    Returns:
        A list of dictionaries, each containing information about detected objects in a corresponding image.
    """

    results = []
    net = cv2.dnn_DetectionModel(model_path, config_path)
    net.setInputSize(320, 320)
    net.setInputScale(1.0 / 127.5)
    net.setInputMean((127.5, 127.5, 127.5))
    net.setInputSwapRB(True)

    with open('coco.names', 'rt') as f:
        classNames = f.read().rstrip('\n').split('\n')

    for image_path in image_paths:
        img = cv2.imread(image_path)
        if img is None:
            print(f"Error loading image: {image_path}")
            continue

        classIds, confs, bbox = net.detect(img, confThreshold=0.7)

        detections = []
        for classId, confidence, box in zip(classIds.flatten(), confs.flatten(), bbox):
            x, y, w, h = box
            detections.append({
                'class': classNames[classId - 1],
                'confidence': float(confidence),
                'bbox': (x, y, w, h)
            })

        results.append({'image_path': image_path, 'detections': detections})

    return results

# Example usage
image_paths = ['/content/car.jpeg', '/content/cat.jpg']
model_path = 'frozen_inference_graph.pb'
config_path = 'ssd_mobilenet_v3_large_coco_2020_01_14.pbtxt'

detection_results = detect_objects_in_images(image_paths, model_path, config_path)
for result in detection_results:
    print(f"Image: {result['image_path']}")
    for detection in result['detections']:
        print(f"  Class: {detection['class']}, Confidence: {detection['confidence']}, Bounding Box: {detection['bbox']}")